## 데이터 전처리 과정
1. 표본이 되는 대여와 반납이 불균형한 (대여가 훨씬 많이 이루어지는) 상위 대여소 30개 추출하기
2. 2023년 월별 대여 데이터로 대여 데이터셋 만들기
3. 2023년 날씨와 미세먼지 데이터 병합하기 (8개의 시간대로 구분하고 각 시간대마다 평균값 구하기)
4. 대여 데이터셋과 날씨/미세먼지 데이터 병합하기
5. 2023년 월별 반납 데이터로 반납 데이터셋 만들고 날씨/미세먼지 데이터와 병합하기

<br>

### 1. 대여가 훨씬 많이 이루어지는 대여소 상위 30개 추출하기
- 모든 대여소를 다루기에는 데이터가 너무 많으므로, 대여와 반납이 불균형한 대여소를 위주로 예측한다. (반납이 많은 곳은 따릉이가 부족할 일이 없으므로)
- 반납-대여 수를 기준으로 상위 30개 대여소를 추출해서 대여소 별 대여건수를 예측한다

In [29]:
# 데이터 전처리에 필요한 라이브러리 불러오기
import numpy as np
import pandas as pd
import datetime as dt
import re

In [233]:
# 23년 월별 대여소별 이용정보 데이터 불러오기
data_2023_1 = pd.read_csv('서울특별시 공공자전거 대여소별 이용정보(월별)_23.1-6.csv', encoding='cp949')
data_2023_2 = pd.read_csv('서울특별시 공공자전거 대여소별 이용정보(월별)_23.7-12.csv', encoding='cp949')

In [234]:
# 2023년 상반기 공공자전거 대여소별 이용정보
# 이용정보는 대여소별 대여건수, 반납건수 확인 가능
data_2023_1

,자치구,대여소명,기준년월,대여건수,반납건수
0,강남구,2301. 현대고등학교 건너편,202301,383,439
1,강남구,2302. 교보타워 버스정류장(신논현역 3번출구 후면),202301,433,446
2,강남구,2303. 논현역 10번출구,202301,648,508
3,강남구,2304. 대현그린타워,202301,76,55
4,강남구,2305. MCM 본사 직영점 앞,202301,172,179
...,...,...,...,...,...
16301,중랑구,4837. 양원지구 힐데스하임 앞,202306,853,827
16302,중랑구,4838.동원사거리 행복오피스텔 앞,202306,900,669
16303,중랑구,4840. 서울시 북부병원 앞,202306,875,897
16304,중랑구,4841. 중화수경공원,202306,1886,2147


In [235]:
data_2023_2

,자치구,대여소명,기준년월,대여건수,반납건수
0,마포구,108. 서교동 사거리,202307,2019,2019
1,양천구,729. 서부식자재마트 건너편,202307,2105,2268
2,양천구,731. 서울시 도로환경관리센터,202307,3975,4042
3,양천구,732. 신월중학교,202307,1284,1260
4,양천구,733. 신정이펜하우스314동,202307,934,451
...,...,...,...,...,...
16376,양천구,722. 경서농협 버스정류장(우리은행신정동지점방향),202312,951,996
16377,양천구,723. SBS방송국,202312,1707,1679
16378,양천구,724. 계남공원 입구 주출입구 좌측,202312,138,94
16379,양천구,725. 양강중학교앞 교차로,202312,299,138


In [236]:
# 2023년 상반기 대여소 목록 확인
data_2023_1['대여소명'].head()

0                  2301. 현대고등학교 건너편
1    2302. 교보타워 버스정류장(신논현역 3번출구 후면)
2                   2303. 논현역 10번출구
3                      2304. 대현그린타워
4                2305. MCM 본사 직영점 앞
Name: 대여소명, dtype: object

In [237]:
# 2023 상하반기 데이터 합치기
data_2023 = pd.concat([data_2023_1, data_2023_2])

# 대여소번호, 기준년월 기준으로 데이터프레임 정렬
data_2023 = data_2023.sort_values(by=['대여소명', '기준년월'])
data_2023 = data_2023.reset_index(drop=True)
data_2023

,자치구,대여소명,기준년월,대여건수,반납건수
0,강동구,1001. 광진교 남단 사거리(천호공원 방면),202301,673,668
1,강동구,1001. 광진교 남단 사거리(천호공원 방면),202302,857,912
2,강동구,1001. 광진교 남단 사거리(천호공원 방면),202303,"1,858","1,833"
3,강동구,1001. 광진교 남단 사거리(천호공원 방면),202304,"2,033","1,918"
4,강동구,1001. 광진교 남단 사거리(천호공원 방면),202305,"2,442","2,443"
...,...,...,...,...,...
32682,성동구,AS센터,202308,53,3
32683,성동구,AS센터,202309,82,4
32684,성동구,AS센터,202310,46,2
32685,성동구,AS센터,202311,2,0


In [238]:
# AS센터, 에이텍, 테스트 대여소는 대여소가 아니므로 제거
data_2023.drop(data_2023[(data_2023['대여소명'] == 'AS센터') | (data_2023['대여소명'] == '9980. 에이텍') | (data_2023['대여소명'] == '9979. 테스트 대여소')].index, inplace=True)

In [239]:
# [대여소명]을 대여소번호와 대여소명으로 분리 : (1001.광진교 남단 사거리 ->  1001, 광진교 남단 사거리)
copy_df = data_2023.copy()
data_2023['대여소명'] = list(map(lambda name : name.split('.')[1].strip(), data_2023['대여소명']))
data_2023['대여소번호'] = list(map(lambda name : name.split('.')[0], copy_df['대여소명']))
data_2023['대여소번호'] = data_2023['대여소번호'].astype(int)

data_2023

,자치구,대여소명,기준년월,대여건수,반납건수,대여소번호
0,강동구,광진교 남단 사거리(천호공원 방면),202301,673,668,1001
1,강동구,광진교 남단 사거리(천호공원 방면),202302,857,912,1001
2,강동구,광진교 남단 사거리(천호공원 방면),202303,"1,858","1,833",1001
3,강동구,광진교 남단 사거리(천호공원 방면),202304,"2,033","1,918",1001
4,강동구,광진교 남단 사거리(천호공원 방면),202305,"2,442","2,443",1001
...,...,...,...,...,...,...
32668,은평구,응암역2번출구 국민은행 앞,202308,4513,5826,996
32669,은평구,응암역2번출구 국민은행 앞,202309,5089,6458,996
32670,은평구,응암역2번출구 국민은행 앞,202310,5757,7230,996
32671,은평구,응암역2번출구 국민은행 앞,202311,3385,4541,996


In [240]:
# 대여건수 중 문자열인 데이터 존재
data_2023.loc[1]['대여건수']

' 857 '

In [241]:
# 정규표현식 사용
import re

# 문자열을 정수로 만들어주는 함수 생성 및 테스트
def string_to_int(num):
    # null 값은 0으로 처리
    if num == ' - ':
        num = 0

    # 대여건수/반납건수가 문자열이라면 특수문자 제거 후 정수로 return
    if type(num) != int:
        num = int(re.compile('\W+').sub('', num))
        
    return num

string_to_int('2,315'), string_to_int(315)

(2315, 315)

In [242]:
# 대여건수가 정수가 아닌 문자열인 것들을 모아서 정수로 만들기
rent = [rental for rental in data_2023['대여건수']]
rent[:10]

[' 673 ',
 ' 857 ',
 ' 1,858 ',
 ' 2,033 ',
 ' 2,442 ',
 ' 2,336 ',
 1850,
 1948,
 2210,
 2693]

In [243]:
rent = list(map(lambda x: string_to_int(x), rent))
rent[:10]

[673, 857, 1858, 2033, 2442, 2336, 1850, 1948, 2210, 2693]

In [244]:
data_2023['대여건수'] = rent
data_2023

,자치구,대여소명,기준년월,대여건수,반납건수,대여소번호
0,강동구,광진교 남단 사거리(천호공원 방면),202301,673,668,1001
1,강동구,광진교 남단 사거리(천호공원 방면),202302,857,912,1001
2,강동구,광진교 남단 사거리(천호공원 방면),202303,1858,"1,833",1001
3,강동구,광진교 남단 사거리(천호공원 방면),202304,2033,"1,918",1001
4,강동구,광진교 남단 사거리(천호공원 방면),202305,2442,"2,443",1001
...,...,...,...,...,...,...
32668,은평구,응암역2번출구 국민은행 앞,202308,4513,5826,996
32669,은평구,응암역2번출구 국민은행 앞,202309,5089,6458,996
32670,은평구,응암역2번출구 국민은행 앞,202310,5757,7230,996
32671,은평구,응암역2번출구 국민은행 앞,202311,3385,4541,996


In [245]:
# 반납도 동일하게 만들어주기
turnin = [turn for turn in data_2023['반납건수']]
turnin = list(map(lambda x: string_to_int(x), turnin))
turnin[:10]

[668, 912, 1833, 1918, 2443, 2330, 1851, 1944, 2223, 2657]

In [246]:
data_2023['반납건수'] = turnin
data_2023.head()

,자치구,대여소명,기준년월,대여건수,반납건수,대여소번호
0,강동구,광진교 남단 사거리(천호공원 방면),202301,673,668,1001
1,강동구,광진교 남단 사거리(천호공원 방면),202302,857,912,1001
2,강동구,광진교 남단 사거리(천호공원 방면),202303,1858,1833,1001
3,강동구,광진교 남단 사거리(천호공원 방면),202304,2033,1918,1001
4,강동구,광진교 남단 사거리(천호공원 방면),202305,2442,2443,1001


In [247]:
# 올바르게 데이터가 변환되었는지 확인
data_2023.dtypes

자치구      object
대여소명     object
기준년월      int64
대여건수      int64
반납건수      int64
대여소번호     int64
dtype: object

In [248]:
# 대여소번호 순으로 정렬
data_2023.sort_values(by=['대여소번호'])

,자치구,대여소명,기준년월,대여건수,반납건수,대여소번호
201,마포구,망원역 1번출구 앞,202301,1406,1638,102
212,마포구,망원역 1번출구 앞,202312,1874,2109,102
211,마포구,망원역 1번출구 앞,202311,3128,3367,102
210,마포구,망원역 1번출구 앞,202310,5225,5474,102
209,마포구,망원역 1번출구 앞,202309,4480,4593,102
...,...,...,...,...,...,...
29425,중구,한국경제,202311,412,443,6055
29426,중구,한국경제,202312,268,269,6055
29559,강서구,월드빌딩 앞,202311,418,387,6171
29560,강서구,월드빌딩 앞,202312,662,640,6171


In [249]:
# 대여소 별 2023년 대여건수, 반납건수 총합이 담긴 데이터프레임 생성
# 대여소 이름은 대여소번호는 같으나 '청담자이아파트앞' -> '청담자이아파트 앞'과 같은 경우가 있으므로 제외
df_2023 = data_2023.groupby(['대여소번호'])[['대여건수', '반납건수']].sum()
# groupby로 묶인 인덱스 초기화
df_2023 = df_2023.reset_index()
df_2023.head()

,대여소번호,대여건수,반납건수
0,102,43079,45033
1,103,35816,38586
2,104,22213,18575
3,105,12625,11335
4,106,36440,35940


In [250]:
# 기존에 중복 문제로 추가하지 못한 대여소명 추가 (최신 이름 기준으로)
df_2023['대여소명'] = [list(data_2023[data_2023['대여소번호'] == n]['대여소명'])[-1] for n in df_2023['대여소번호']]
df_2023.head()

,대여소번호,대여건수,반납건수,대여소명
0,102,43079,45033,망원역 1번출구 앞
1,103,35816,38586,망원역 2번출구 앞
2,104,22213,18575,합정역 1번출구 앞
3,105,12625,11335,합정역 5번출구 앞
4,106,36440,35940,합정역 7번출구 앞


In [251]:
# 대여소번호 기준 중복값 확인
df_2023[df_2023.duplicated(subset='대여소번호')]

,대여소번호,대여건수,반납건수,대여소명


In [252]:
# 대여반납 불균형을 확인하기 위한 반납-대여건수 변수 추가
df_2023['반납-대여건수'] = df_2023['반납건수'] - df_2023['대여건수']
df_2023.sort_values(by='반납-대여건수', inplace=True)

In [253]:
# 표본으로 사용할 상위 30개 csv로 내보내기
df_2023[:30].to_csv('[상위 30개]2023년 따릉이 대여소별 대여반납건수(반납건수-대여건수 내림차순)_240529.csv', encoding='cp949', index=False)

<br>

### 2. 2023년 월별 대여 데이터로 대여 데이터셋 만들기

In [254]:
# 1~12월 대여소별 데이터 불러오기
rental_2301 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2301.csv', encoding='cp949')
rental_2302 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2302.csv', encoding='cp949')
rental_2303 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2303.csv', encoding='cp949')
rental_2304 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2304.csv', encoding='cp949')
rental_2305 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2305.csv', encoding='cp949')
rental_2306 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2306.csv', encoding='cp949')
rental_2307 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2307.csv', encoding='cp949')
rental_2308 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2308.csv', encoding='cp949')
rental_2309 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2309.csv', encoding='cp949')
rental_2310 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2310.csv', encoding='cp949')
rental_2311 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2311.csv', encoding='cp949')
rental_2312 = pd.read_csv('서울특별시 공공자전거 이용정보(시간대별)_2312.csv', encoding='cp949')

# 표본 데이터 불러오기
sample = pd.read_csv('[상위 30개]2023년 따릉이 대여소별 대여반납건수(반납건수-대여건수 내림차순)_240529.csv', encoding='cp949')

In [255]:
# 데이터 확인 (1월 데이터를 예로)
rental_2301.head()

,대여일자,대여시간,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분)
0,2023-01-01,0,433,433. 을지로입구역 2번출구,정기권,NaN,~10대,1,44.24,0.42,1831.45,8
1,2023-01-01,0,2032,2032. 이수역 11번출구쪽,정기권,NaN,~10대,1,7.11,0.06,276.06,4
2,2023-01-01,0,744,744. 신목동역 2번 출구,정기권,NaN,20대,1,25.05,0.24,1020.46,7
3,2023-01-01,0,1029,1029. 성내어울터,정기권,NaN,20대,1,34.05,0.20,859.74,4
4,2023-01-01,0,1153,"1153. 발산역 1번, 9번 인근 대여소",정기권,NaN,20대,2,141.49,1.31,5675.10,78


In [256]:
# 데이터 확인 (1월 데이터를 예로)
rental_2301.dtypes

대여일자        object
대여시간         int64
대여소번호        int64
대여소명        object
대여구분코드      object
성별          object
연령대코드       object
이용건수         int64
운동량         object
탄소량         object
이동거리(M)    float64
이용시간(분)      int64
dtype: object

In [257]:
# 표본 데이터 확인
sample

,대여소번호,대여건수,반납건수,대여소명,반납-대여건수
0,574,39597,22567,아차산역4번출구,-17030
1,711,17370,2900,신일해피트리아파트 앞,-14470
2,3513,30022,19156,상왕십리역 1번출구,-10866
3,623,27664,16886,서울시립대 정문 앞 B,-10778
4,1126,27603,16930,우장산역 1번출구옆(우장산아이파크105동앞),-10673
5,207,113094,102439,여의나루역 1번출구 앞,-10655
6,584,42593,31953,광진광장 교통섬,-10640
7,1692,27991,18358,온곡초교 교차로,-9633
8,2712,13681,4377,화곡터널입구교차로,-9304
9,4591,10961,2570,신길자이아파트,-8391


In [258]:
# 표본으로 사용할 대여소번호 목록
sample_num = sorted(list(sample['대여소번호']))

In [259]:
# 필요하지 않은 변수는 추후 드랍할 예정이지만 다른 데이터프레임과 형식을 통일 하기 위해 임의로 수정
# 12월의 이동거리 변수만 열 이름이 '이동거리(M)'이 아닌 '이동거리'로 되어있음, 이용시간(분) 변수는 없음
rental_2312.rename(columns= {'이동거리' : '이동거리(M)'}, inplace=True)
rental_2312['이용시간(분)'] = 0
rental_2312

,대여일자,대여시간,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분)
0,2023-12-01,0,3517,3517. 용마사거리,정기권,NaN,~10대,1,6.88,0.40,1738.19,0
1,2023-12-01,0,1569,1569.수유역2번출구,정기권,NaN,~10대,1,78.37,0.61,2638.76,0
2,2023-12-01,0,736,736. 오솔길공원,정기권,NaN,20대,1,40.91,0.32,1377.52,0
3,2023-12-01,0,740,740. 으뜸공원,정기권,NaN,20대,1,90.50,0.53,2285.26,0
4,2023-12-01,0,505,505. 자양사거리 광진아크로텔 앞,정기권,NaN,20대,1,21.38,0.21,900.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1954954,2023-12-31,23,1526,1526. 강북종합시장 앞,정기권,M,기타,1,37.36,0.26,1110.00,0
1954955,2023-12-31,23,1986,1986. 태평양물산빌딩,정기권,M,기타,1,20.29,0.18,776.29,0
1954956,2023-12-31,23,293,293. 충북 미래관,정기권,M,기타,1,14.67,0.12,529.22,0
1954957,2023-12-31,23,779,779. 신정1동 주민센터 앞,정기권,M,기타,1,87.44,0.64,2760.00,0


In [260]:
# 월별 시간대별 이용정보를 전처리하기 쉽도록 정제하는 함수
# 23년 1월 데이터를 활용한 02-1 예제 파일을 확인하면 가공 과정을 상세히 확인 가능

def convert_rental_df(rental_month, sample_num = sample_num):
    df = rental_month.copy()
    
    #샘플 대여소만 추출
    df = df[df['대여소번호'].isin(sample_num)]
    #필요없는 변수 제거
    df.drop(['대여구분코드', '성별', '연령대코드', '운동량', '탄소량', '이동거리(M)', '이용시간(분)'], axis=1, inplace=True)
    
    '''대여일 및 대여시간 변수 생성'''
    df['대여일자'] = pd.to_datetime(df['대여일자'])
    df = df[['대여일자', '대여시간', '대여소번호', '대여소명']]
    
    ''' 잘못된 변수명 변경 : 대여소명 (233. 정류소이름) -> (정류소이름) 으로 변경'''
    # '.'을 기준으로 오른쪽이 대여소 이름이므로, 해당 부분 추출 후 공백 제거
    df['대여소명'] = list(map(lambda name : name.split('.')[1].strip(), df['대여소명']))
    
    '''대여건수 변수 생성'''
    df['대여건수'] = 1
    df = df.groupby(['대여일자', '대여시간', '대여소번호', '대여소명'])['대여건수'].sum()
    df = df.reset_index()
    
    '''대여일자 - 대여시간 - 대여소번호 별로 정렬'''
    df.sort_values(by=['대여일자', '대여시간', '대여소번호'], inplace=True)
    
    # 인덱스 초기화
    df = df.reset_index(drop=True)
    
    return df

In [261]:
# 2023 1월 데이터를 예로 확인
# 대여건수가 있는 대여시간만 기록되어 있음
convert_rental_df(rental_2301).head()

,대여일자,대여시간,대여소번호,대여소명,대여건수
0,2023-01-01,0,207,여의나루역 1번출구 앞,2
1,2023-01-01,0,213,KT앞,1
2,2023-01-01,0,434,신당 래미안 버스정류장,1
3,2023-01-01,0,474,동대문역사문화공원역 1번출구 뒤편,3
4,2023-01-01,0,623,서울시립대 정문 앞,1


In [262]:
# 2023년 각 월별 대여 데이터 리스트 생성
rental_2023 = [rental_2301, rental_2302, rental_2303, rental_2304, rental_2305, rental_2306, rental_2307, rental_2308, rental_2309, rental_2310, rental_2311, rental_2312]

In [263]:
# 월별 대여 데이터들을 위에 작성한 convert_rental_df 대여소-시간대별 대여 데이터로 가공한 station_dfs 리스트에 담기
station_dfs = list(map(lambda x: convert_rental_df(x), rental_2023))

# 월별 데이터를 모두 합쳐서 1년치로 만들기
station_df_2023 = pd.concat(station_dfs)

In [265]:
# 2023년 대여소별 대여 데이터프레임이 만들어짐
# 하지만 대여건수가 0인 경우는 포함되지 않아 해당 경우도 추가해야 함
# 결과적으로 행의 수, 기대값은 365 (일) * 24 (시간) * 30 (대여소) = 262800개가 되어야 함
station_df_2023 = station_df_2023.reset_index()
station_df_2023.drop(['index', 'level_0'], axis=1, inplace=True)
station_df_2023.tail()

,대여일자,대여시간,대여소번호,대여소명,대여건수
166122,2023-12-31,23,574,아차산역4번출구,2
166123,2023-12-31,23,584,광진광장 교통섬,4
166124,2023-12-31,23,1175,대한항공 인력개발센터,1
166125,2023-12-31,23,3513,상왕십리역 1번출구,1
166126,2023-12-31,23,4591,신길자이아파트,1


In [266]:
import numpy as np
from itertools import product

# Extract unique sets of 대여소번호 and 대여소명
unique_stations = sample[['대여소번호', '대여소명']]

# 2023년 모든 날짜 생성하기
all_dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D').strftime('%Y-%m-%d')

# 시간대 생성하기
all_hours = np.arange(24)

# 대여소별 2023년 날짜-시간대 데이터가 있는 데이터프레임 생성
all_combinations = pd.DataFrame(
    [(date, hour, station['대여소번호'], station['대여소명']) 
     for date in all_dates 
     for hour in all_hours 
     for _, station in unique_stations.iterrows()], 
    columns=['대여일자', '대여시간', '대여소번호', '대여소명']
)

# 대여일자 data type object -> datetimes로 변경
all_combinations['대여일자'] = pd.to_datetime(all_combinations['대여일자'])

# all_combinations 데이터 프레임의 행 수 확인 (262800이어야 함)
num_combinations = len(all_combinations)
expected_combinations = 24 * 365 * len(unique_stations)
print(f"Number of combinations: {num_combinations}, Expected: {expected_combinations}")

# 기대값 (262800)과 일치하는지 확인
assert num_combinations == expected_combinations, f"Mismatch: {num_combinations} != {expected_combinations}"

Number of combinations: 262800, Expected: 262800


In [267]:
all_combinations.dtypes

대여일자     datetime64[ns]
대여시간              int64
대여소번호             int64
대여소명             object
dtype: object

In [268]:
# 원본 데이터를 all_combinations 데이터 프레임과 병합
full_data_2023 = pd.merge(all_combinations, station_df_2023, on=['대여일자', '대여시간', '대여소번호', '대여소명'], how='left')

# 대여건수 null값은 대여건수가 없는 것이므로 0으로 채우기
full_data_2023['대여건수'] = full_data_2023['대여건수'].fillna(0)

# 데이터 확인
full_data_2023.head()

,대여일자,대여시간,대여소번호,대여소명,대여건수
0,2023-01-01,0,574,아차산역4번출구,0.0
1,2023-01-01,0,711,신일해피트리아파트 앞,0.0
2,2023-01-01,0,3513,상왕십리역 1번출구,0.0
3,2023-01-01,0,623,서울시립대 정문 앞 B,0.0
4,2023-01-01,0,1126,우장산역 1번출구옆(우장산아이파크105동앞),0.0


In [269]:
# 대여건수 데이터타입 정수로 변환
full_data_2023 = full_data_2023.astype({'대여건수' : int})

In [270]:
# 대여소별로 확인 가능하도록 대여소번호를 최우선 기준으로 sort
full_data_2023.sort_values(by=['대여소번호', '대여일자', '대여시간'], inplace=True)

In [272]:
# 요일 변수 생성 (추후 평일구분 변수로 변환 예정)
full_data_2023['요일'] = [date.weekday() for date in full_data_2023['대여일자']]
full_data_2023.head()

,대여일자,대여시간,대여소번호,대여소명,대여건수,요일
22,2023-01-01,0,162,봉원고가차도 밑,0,6
52,2023-01-01,1,162,봉원고가차도 밑,2,6
82,2023-01-01,2,162,봉원고가차도 밑,0,6
112,2023-01-01,3,162,봉원고가차도 밑,0,6
142,2023-01-01,4,162,봉원고가차도 밑,0,6


In [273]:
# 평일은 '평일', 평일이 아니면 '평일 외'로 값 변경
full_data_2023['요일'].replace(to_replace=[0, 1, 2, 3, 4],value='평일', inplace=True)
full_data_2023['요일'].replace(to_replace=[5, 6],value='평일 외', inplace=True)

# 열 이름도 변경하기
full_data_2023.rename(columns={'요일':'평일구분'}, inplace=True)

full_data_2023.head()

,대여일자,대여시간,대여소번호,대여소명,대여건수,평일구분
22,2023-01-01,0,162,봉원고가차도 밑,0,평일 외
52,2023-01-01,1,162,봉원고가차도 밑,2,평일 외
82,2023-01-01,2,162,봉원고가차도 밑,0,평일 외
112,2023-01-01,3,162,봉원고가차도 밑,0,평일 외
142,2023-01-01,4,162,봉원고가차도 밑,0,평일 외


In [274]:
# 평일인 공휴일 '평일' -> '평일 외'로 변경하기 위해 공휴일 데이터 불러오기
holiday_2023 = pd.read_csv('2023_평일인공휴일.csv', encoding = 'utf-8')

# 마지막 행 null이므로 제거
holiday_2023.drop([14], axis=0, inplace=True)
holiday_2023

,공휴일 이름,날짜,23년도 당시 요일
0,설날 연휴(4-3),2023-01-24,월요일
1,설날 연휴(4-4),2023-01-25,화요일
2,삼일절,2023-03-01,수요일
3,노동절,2023-05-01,월요일
4,어린이날,2023-05-05,금요일
5,석가탄신일 대체 공휴일,2023-05-29,월요일
6,현충일,2023-06-06,화요일
7,광복절,2023-08-15,화요일
8,추석연휴(4-1),2023-09-28,목요일
9,추석연휴(4-2),2023-09-29,금요일


In [275]:
# 공휴일 csv의 날짜 문자열 -> datetime 형태로 변환
holiday_2023['날짜'] = pd.to_datetime(holiday_2023['날짜'])
holiday_2023['날짜']

0    2023-01-24
1    2023-01-25
2    2023-03-01
3    2023-05-01
4    2023-05-05
5    2023-05-29
6    2023-06-06
7    2023-08-15
8    2023-09-28
9    2023-09-29
10   2023-10-02
11   2023-10-03
12   2023-10-09
13   2023-12-25
Name: 날짜, dtype: datetime64[ns]

In [276]:
# 데이터에서 공휴일인 평일 '평일 외'로 변경
for holiday in holiday_2023['날짜']:
    # 공휴일 (평일이 아닌데 평일로 분류된) 행들의 index 
    not_week_index = full_data_2023[full_data_2023['대여일자'] == holiday].index
    
    # 공휴일 행의 '평일구분'을 '평일' -> '평일 외'로 변경
    full_data_2023.loc[not_week_index, '평일구분'] = '평일 외'

In [278]:
# 정상적으로 변경된 것 확인 : '2023-09-29'는 원래 수요일이나 추석연휴 였기 때문에 평일 외로 변경
full_data_2023[full_data_2023['대여일자'] == '2023-09-28'].head()

,대여일자,대여시간,대여소번호,대여소명,대여건수,평일구분
194422,2023-09-28,0,162,봉원고가차도 밑,0,평일 외
194452,2023-09-28,1,162,봉원고가차도 밑,0,평일 외
194482,2023-09-28,2,162,봉원고가차도 밑,0,평일 외
194512,2023-09-28,3,162,봉원고가차도 밑,0,평일 외
194542,2023-09-28,4,162,봉원고가차도 밑,0,평일 외


In [310]:
# 2023년 대여소별 공공자전거 대여건수 데이터 csv로 내보내기
full_data_2023.to_csv('2023년 대여소별 공공자전거 대여건수(시간대별)_0529.csv', index=False, encoding='cp949')

<br>

### 3. 2023년 날씨와 미세먼지 데이터 병합하기

In [295]:
# 2023년 기온데이터와 미세먼지 데이터 불러오기
weather_2023 = pd.read_csv('2023_기온데이터.csv', encoding='cp949')
dust_2023 = pd.read_csv('2023_미세먼지.csv', encoding='cp949')

In [296]:
# 필요 없는 변수 제거
weather_2023.drop(['지점', '지점명'], axis=1, inplace=True)
dust_2023.drop(['지점', '지점명'], axis=1, inplace=True)

In [297]:
weather_2023.head()

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
0,2023-01-01 00:00,0.9,NaN,1.4,72,NaN
1,2023-01-01 01:00,1.5,NaN,1.9,71,NaN
2,2023-01-01 02:00,1.5,NaN,1.9,72,NaN
3,2023-01-01 03:00,1.6,NaN,1.6,74,NaN
4,2023-01-01 04:00,1.5,NaN,1.4,74,NaN


In [298]:
dust_2023.head()

,일시,1시간평균 미세먼지농도(㎍/㎥)
0,2023-01-01 00:00,63
1,2023-01-01 01:00,67
2,2023-01-01 02:00,84
3,2023-01-01 03:00,82
4,2023-01-01 04:00,87


In [299]:
# 미세먼지 데이터에 결측치 존재 (1년은 8760시간 - 일부 시간 미세먼지 데이터 없음)
# 미세먼지는 min 2 ~ max 400
len(dust_2023), len(weather_2023)

(8372, 8760)

In [300]:
# 기온+미세먼지 데이터 합치기 : 기온데이터를 기준으로
weather_dust_2023 = pd.merge(weather_2023, dust_2023, on='일시', how = 'left')
weather_dust_2023.head()

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),1시간평균 미세먼지농도(㎍/㎥)
0,2023-01-01 00:00,0.9,NaN,1.4,72,NaN,63.0
1,2023-01-01 01:00,1.5,NaN,1.9,71,NaN,67.0
2,2023-01-01 02:00,1.5,NaN,1.9,72,NaN,84.0
3,2023-01-01 03:00,1.6,NaN,1.6,74,NaN,82.0
4,2023-01-01 04:00,1.5,NaN,1.4,74,NaN,87.0


In [301]:
# 날짜 / 시간 분리 작업
dates = []
times = []

for idx in weather_dust_2023.index:
    date = weather_dust_2023['일시'].loc[idx][:10]
    time = int(weather_dust_2023['일시'].loc[idx][-5:-3])
    
    dates.append(date)
    times.append(time)

In [302]:
# '일시' -> '날짜' , '시간대'로 분리 : '2023-01-01 01:00' -> '2023-01-01', 1
dates = [date[:10] for date in weather_dust_2023['일시']]
times = [int(time[-5:-3]) for time in weather_dust_2023['일시']]

In [303]:
# 날짜, 시간대 열 생성
weather_dust_2023.insert(loc=1, column='날짜', value=dates)
weather_dust_2023.insert(loc=2, column='시간대', value=times)

# 기존 '일시' 열 삭제
weather_dust_2023.drop('일시', axis=1, inplace=True)

In [304]:
weather_dust_2023.head()

,날짜,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),1시간평균 미세먼지농도(㎍/㎥)
0,2023-01-01,0,0.9,NaN,1.4,72,NaN,63.0
1,2023-01-01,1,1.5,NaN,1.9,71,NaN,67.0
2,2023-01-01,2,1.5,NaN,1.9,72,NaN,84.0
3,2023-01-01,3,1.6,NaN,1.6,74,NaN,82.0
4,2023-01-01,4,1.5,NaN,1.4,74,NaN,87.0


In [305]:
# 결측치 채우기 : 미세먼지농도와 풍속 null값은 이전 시간대의 것으로 채우기
weather_dust_2023['1시간평균 미세먼지농도(㎍/㎥)'] = weather_dust_2023['1시간평균 미세먼지농도(㎍/㎥)'].fillna(method='ffill')
weather_dust_2023['풍속(m/s)'] = weather_dust_2023['풍속(m/s)'].fillna(method='ffill')

# 결측치 채우기 : 강수량, 적설의 null값은 비나 눈이 오지 않은 것이므로 0으로 채우기
weather_dust_2023['강수량(mm)'] = weather_dust_2023['강수량(mm)'].fillna(0)
weather_dust_2023['적설(cm)'] = weather_dust_2023['적설(cm)'].fillna(0)

In [306]:
# 결측치가 제거된 것 확인
weather_dust_2023.isnull().sum()

날짜                   0
시간대                  0
기온(°C)               0
강수량(mm)              0
풍속(m/s)              0
습도(%)                0
적설(cm)               0
1시간평균 미세먼지농도(㎍/㎥)    0
dtype: int64

In [307]:
# 2023년 날씨+미세먼지 병합 데이터 csv로 내보내기
weather_dust_2023.to_csv('[결측치 제거]2023년 날씨+미세먼지정보.csv', index=False, encoding='cp949')

In [54]:
weather = pd.read_csv('[결측치 제거]2023년 날씨+미세먼지정보.csv', encoding='cp949')

In [55]:
# 8개의 시간대로 날씨 데이터 분할
a = weather[weather['시간대'].isin([2, 3, 4])] #2-4시
b = weather[weather['시간대'].isin([5, 6, 7])] #5-7시
c = weather[weather['시간대'].isin([8, 9, 10])] #8-10시
d = weather[weather['시간대'].isin([11, 12, 13])] #11-13시
e = weather[weather['시간대'].isin([14, 15, 16])] #14-16시
f = weather[weather['시간대'].isin([17, 18, 19])] #17-19시
g = weather[weather['시간대'].isin([20, 21, 22])] #20-22시
h = weather[weather['시간대'].isin([23, 0, 1])] #23-1시

In [56]:
# 날짜 별로 그룹핑해서 각 변수의 평균값 구하고, 시간대 새로 지정
a = a.groupby('날짜').mean().reset_index()
a['시간대'] = 1
b = b.groupby('날짜').mean().reset_index()
b['시간대'] = 2
c = c.groupby('날짜').mean().reset_index()
c['시간대'] = 3
d = d.groupby('날짜').mean().reset_index()
d['시간대'] = 4
e = e.groupby('날짜').mean().reset_index()
e['시간대'] = 5
f = f.groupby('날짜').mean().reset_index()
f['시간대'] = 6
g = g.groupby('날짜').mean().reset_index()
g['시간대'] = 7
h = h.groupby('날짜').mean().reset_index()
h['시간대'] = 8

In [57]:
# 각각의 시간대 별 데이터 병합
mean_weather = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
mean_weather.rename(columns = {'1시간평균 미세먼지농도(㎍/㎥)':'미세먼지농도(㎍/㎥)'}, inplace=True)
mean_weather

,날짜,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥)
0,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333
1,2023-01-02,1,-6.133333,0.000000,1.900000,52.000000,0.000000,18.666667
2,2023-01-03,1,-7.966667,0.000000,1.466667,59.333333,0.000000,30.333333
3,2023-01-04,1,-5.466667,0.000000,1.833333,55.000000,0.000000,31.000000
4,2023-01-05,1,-3.900000,0.000000,1.100000,65.333333,0.000000,41.666667
...,...,...,...,...,...,...,...,...
2915,2023-12-27,8,0.566667,0.000000,0.733333,81.333333,0.000000,80.000000
2916,2023-12-28,8,0.800000,0.000000,0.900000,75.666667,0.000000,59.666667
2917,2023-12-29,8,1.233333,0.000000,1.133333,68.333333,0.000000,34.333333
2918,2023-12-30,8,0.500000,0.000000,2.066667,85.000000,3.266667,34.333333


<br>

### 4. 대여 데이터셋과 날씨/미세먼지 데이터 병합하기

In [50]:
station_rental_2023 = pd.read_csv('2023년 대여소별 공공자전거 대여건수(시간대별)_0529.csv', encoding='cp949')

In [51]:
# 8개의 시간대로 따릉이 데이터 분할
station_rental_2023['시간대'] = station_rental_2023['대여시간'].map({2:1, 3:1, 4:1, 5:2, 6:2, 7:2, 8:3, 9:3, 10:3, 11:4, 12:4, 13:4, 
                                                                 14:5, 15:5, 16:5, 17:6, 18:6, 19:6, 20:7, 21:7, 22:7, 23:8, 0:8, 1:8})

station_rental_2023 = station_rental_2023.drop('대여시간', axis=1) #대여시간 열 삭제

In [52]:
# 시간대별 대여건수 추출
grouped = station_rental_2023.groupby(['대여일자', '시간대', '평일구분', '대여소번호', '대여소명'])
station_rental_2023 = grouped['대여건수'].agg('sum').reset_index()
station_rental_2023

,대여일자,시간대,평일구분,대여소번호,대여소명,대여건수
0,2023-01-01,1,평일 외,162,봉원고가차도 밑,0
1,2023-01-01,1,평일 외,207,여의나루역 1번출구 앞,2
2,2023-01-01,1,평일 외,213,KT앞,2
3,2023-01-01,1,평일 외,434,신당 래미안 버스정류장,1
4,2023-01-01,1,평일 외,474,동대문역사문화공원역 1번출구 뒤편,5
...,...,...,...,...,...,...
87595,2023-12-31,8,평일 외,3758,서울남부출입국관리소,0
87596,2023-12-31,8,평일 외,4009,월계프라자,0
87597,2023-12-31,8,평일 외,4221,솔리스타오피스텔(공덕동주민센터),0
87598,2023-12-31,8,평일 외,4591,신길자이아파트,1


In [58]:
mean_weather

,날짜,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥)
0,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333
1,2023-01-02,1,-6.133333,0.000000,1.900000,52.000000,0.000000,18.666667
2,2023-01-03,1,-7.966667,0.000000,1.466667,59.333333,0.000000,30.333333
3,2023-01-04,1,-5.466667,0.000000,1.833333,55.000000,0.000000,31.000000
4,2023-01-05,1,-3.900000,0.000000,1.100000,65.333333,0.000000,41.666667
...,...,...,...,...,...,...,...,...
2915,2023-12-27,8,0.566667,0.000000,0.733333,81.333333,0.000000,80.000000
2916,2023-12-28,8,0.800000,0.000000,0.900000,75.666667,0.000000,59.666667
2917,2023-12-29,8,1.233333,0.000000,1.133333,68.333333,0.000000,34.333333
2918,2023-12-30,8,0.500000,0.000000,2.066667,85.000000,3.266667,34.333333


In [59]:
# 데이터프레임 병합을 위해 열 이름 통일
mean_weather.rename(columns= {'날짜' : '대여일자'}, inplace=True)

In [60]:
### 날씨와 따릉이 데이터 병합
rental = pd.merge(mean_weather, station_rental_2023, on=['대여일자', '시간대'], how='left')
rental

,대여일자,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥),평일구분,대여소번호,대여소명,대여건수
0,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,162,봉원고가차도 밑,0
1,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,207,여의나루역 1번출구 앞,2
2,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,213,KT앞,2
3,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,434,신당 래미안 버스정류장,1
4,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,474,동대문역사문화공원역 1번출구 뒤편,5
...,...,...,...,...,...,...,...,...,...,...,...,...
87595,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,3758,서울남부출입국관리소,0
87596,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4009,월계프라자,0
87597,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4221,솔리스타오피스텔(공덕동주민센터),0
87598,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4591,신길자이아파트,1


In [61]:
### '월' 변수 만들기
rental['대여일자'] = pd.to_datetime(rental['대여일자'])
rental['대여월'] = rental['대여일자'].dt.month

### 평일구분 -> 평일로 바꾸기 (평일이면 1, 평일 아니면 0)
rental['평일'] = 0
rental['평일'] = np.where(rental['평일구분'] == '평일', 1, 0) 

### 필요없는 열 삭제
rental = rental.drop(['평일구분', '대여일자', '대여소명'], axis=1)

rental

,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥),대여소번호,대여건수,대여월,평일
0,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,162,0,1,0
1,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,207,2,1,0
2,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,213,2,1,0
3,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,434,1,1,0
4,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,474,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
87595,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,3758,0,12,0
87596,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,4009,0,12,0
87597,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,4221,0,12,0
87598,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,4591,1,12,0


In [62]:
### 데이터 파일로 저장
rental.to_csv('2023년_따릉이대여_날씨_3시간별.csv', encoding='cp949')

<br>

### 5. 2023년 월별 반납 데이터로 반납 데이터셋 만들고 날씨/미세먼지 데이터와 병합하기

In [1]:
### 1월~6월 따릉이 반납 데이터 생성 함수
def jan_to_jun(df, month):
    
    # 대여-반납 차이 상위 30개 대여소만 추출
    df = df.drop(df[df['반납대여소번호'] == '\\N'].index)
    df['반납대여소번호'] = df['반납대여소번호'].astype(int)
    df = df[df['반납대여소번호'].isin([574, 711, 3513, 623, 1126, 207, 584, 1692, 2712, 4591, 
                                   2405, 3758, 434, 2349, 1924, 1430, 1199, 213, 1428, 497, 4621, 
                                   4221, 162, 725, 3642, 474, 4009, 1854, 720, 1175])]

    # 반납 관련 변수 추출
    df = df[['반납일시', '반납대여소번호', '반납대여소명']]

    # 반납일 및 반납시간 변수 생성
    df['반납일시'] = pd.to_datetime(df['반납일시'])
    df['반납일'] = df['반납일시'].dt.strftime('%Y-%m-%d')
    df['반납시간'] = df['반납일시'].dt.hour
    df = df[['반납일', '반납시간', '반납대여소번호', '반납대여소명']]

    # 다음 달 1일 반납 데이터 삭제
    date = '2023-0{}-01'.format(month)
    df = df[df['반납일'] != date]

    # 반납건수 변수 생성
    df['반납건수'] = 1
    df = df.groupby(['반납일', '반납시간', '반납대여소번호', '반납대여소명'])['반납건수'].sum()
    df = df.reset_index()

    return df

In [2]:
### 7월~8월 따릉이 반납 데이터 생성 함수
def jul_to_aug(df, month):

    # 일반자전거인 경우만 추출
    df = df[df['자전거구분'] == '일반자전거']
    
    # 대여-반납 차이 상위 30개 대여소만 추출
    df = df.drop(df[df['반납대여소번호'] == '\\N'].index)
    df['반납대여소번호'] = df['반납대여소번호'].astype(int)
    df = df[df['반납대여소번호'].isin([574, 711, 3513, 623, 1126, 207, 584, 1692, 2712, 4591, 
                                   2405, 3758, 434, 2349, 1924, 1430, 1199, 213, 1428, 497, 4621, 
                                   4221, 162, 725, 3642, 474, 4009, 1854, 720, 1175])]

    # 반납 관련 변수 추출
    df = df[['반납일시', '반납대여소번호', '반납대여소명']]

    # 반납일 및 반납시간 변수 생성
    df['반납일시'] = pd.to_datetime(df['반납일시'])
    df['반납일'] = df['반납일시'].dt.strftime('%Y-%m-%d')
    df['반납시간'] = df['반납일시'].dt.hour
    df = df[['반납일', '반납시간', '반납대여소번호', '반납대여소명']]

    # 다음 달 1일 반납 데이터 삭제
    date = '2023-0{}-01'.format(month)
    df = df[df['반납일'] != date]

    # 반납건수 변수 생성
    df['반납건수'] = 1
    df = df.groupby(['반납일', '반납시간', '반납대여소번호', '반납대여소명'])['반납건수'].sum()
    df = df.reset_index()

    return df

In [3]:
### 9~12월 따릉이 반납 데이터 생성 함수 
def sep_to_dec(df, month):

    # 일반자전거인 경우만 추출
    df = df[df['자전거구분'] == '일반자전거']
    
    # 대여-반납 차이 상위 30개 대여소만 추출
    df = df.drop(df[df['반납대여소번호'] == '\\N'].index)
    df['반납대여소번호'] = df['반납대여소번호'].astype(int)
    df = df[df['반납대여소번호'].isin([574, 711, 3513, 623, 1126, 207, 584, 1692, 2712, 4591, 
                                   2405, 3758, 434, 2349, 1924, 1430, 1199, 213, 1428, 497, 4621, 
                                   4221, 162, 725, 3642, 474, 4009, 1854, 720, 1175])]

    # 반납 관련 변수 추출
    df = df[['반납일시', '반납대여소번호', '반납대여소명']]

    # 반납일 및 반납시간 변수 생성
    df['반납일시'] = pd.to_datetime(df['반납일시'])
    df['반납일'] = df['반납일시'].dt.strftime('%Y-%m-%d')
    df['반납시간'] = df['반납일시'].dt.hour
    df = df[['반납일', '반납시간', '반납대여소번호', '반납대여소명']]

    # 다음 달 1일 반납 데이터 삭제
    date = '2023-{}-01'.format(month)
    df = df[df['반납일'] != date]

    # 반납건수 변수 생성
    df['반납건수'] = 1
    df = df.groupby(['반납일', '반납시간', '반납대여소번호', '반납대여소명'])['반납건수'].sum()
    df = df.reset_index()

    return df

In [6]:
jan = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2301.csv", encoding="cp949")
jan = jan_to_jun(jan, 2)

In [7]:
feb = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2302.csv", encoding="cp949")
feb = jan_to_jun(feb, 3)

In [8]:
mar = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2303.csv", encoding="cp949")
mar = jan_to_jun(mar, 4)

In [9]:
apr = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2304.csv", encoding="cp949")
apr = jan_to_jun(apr, 5)

In [10]:
may = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2305.csv", encoding="cp949")
may = jan_to_jun(may, 6)

In [11]:
jun = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2306.csv", encoding="cp949")
jun = jan_to_jun(jun, 7)

In [12]:
jul = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2307.csv", encoding="cp949")
jul = jul_to_aug(jul, 8)

In [13]:
aug = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2308.csv", encoding="cp949")
aug = jul_to_aug(aug, 9)

In [14]:
sep = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2309.csv", encoding="cp949")
sep = sep_to_dec(sep, 10)

In [15]:
oct = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2310.csv", encoding="cp949")
oct = sep_to_dec(oct, 11)

In [16]:
nov = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2311.csv", encoding="cp949")
nov = sep_to_dec(nov, 12)

In [17]:
dec = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2312.csv", encoding="cp949")
dec = sep_to_dec(dec, 0o1)
dec = dec[dec['반납일'] != '2024-01-01']

In [18]:
bike_return = pd.concat([jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec], ignore_index=True)
bike_return

,반납일,반납시간,반납대여소번호,반납대여소명,반납건수
0,2023-01-01,0,207,여의나루역 1번출구 앞,2
1,2023-01-01,0,474,동대문역사문화공원역 1번출구 뒤편,1
2,2023-01-01,0,574,아차산역4번출구,2
3,2023-01-01,0,725,양강중학교앞 교차로,1
4,2023-01-01,0,1126,우장산역 1번출구옆(우장산아이파크105동앞),1
...,...,...,...,...,...
128267,2023-12-31,23,574,아차산역4번출구,2
128268,2023-12-31,23,584,광진광장 교통섬,3
128269,2023-12-31,23,711,신일해피트리아파트 앞,2
128270,2023-12-31,23,1175,대한항공 인력개발센터,1


In [20]:
# 1: 2-4시, 2: 5-7시, 3: 8-10시, 4: 11-13시, 5: 14-16시, 6: 17-19시, 7: 20-22시, 8: 23-1시
bike_return['시간대'] = bike_return['반납시간'].map({2:1, 3:1, 4:1, 5:2, 6:2, 7:2, 8:3, 9:3, 10:3, 11:4, 12:4, 13:4, 
                                                 14:5, 15:5, 16:5, 17:6, 18:6, 19:6, 20:7, 21:7, 22:7, 23:8, 0:8, 1:8})

bike_return = bike_return.drop(['반납시간', '반납대여소명'], axis=1)

In [21]:
# 반납건수 변수 생성
grouped = bike_return.groupby(['반납일', '시간대', '반납대여소번호'])
bike_return = grouped['반납건수'].agg('sum').reset_index()
bike_return

,반납일,시간대,반납대여소번호,반납건수
0,2023-01-01,1,162,1
1,2023-01-01,1,207,4
2,2023-01-01,1,213,2
3,2023-01-01,1,474,1
4,2023-01-01,1,497,2
...,...,...,...,...
63098,2023-12-31,8,711,2
63099,2023-12-31,8,1175,1
63100,2023-12-31,8,1199,1
63101,2023-12-31,8,1428,1


In [22]:
### 반납이 0인 경우 데이터가 존재하지 않기 때문에 대여 데이터와 합치면서 0값을 만들어 주기
# 대여 데이터와 merge를 위해 변수명 변경
bike_return.rename(columns = {'반납일':'대여일자', '반납대여소번호':'대여소번호'}, inplace=True)

# merge 기준이 될 열의 타입 변경 
bike_return = bike_return.astype({'대여일자':'object', '시간대':'int64', '대여소번호':'int64'})
bike_return.dtypes

대여일자     object
시간대       int64
대여소번호     int64
반납건수      int64
dtype: object

In [23]:
# 대여 데이터 로드
bike_rental = pd.read_csv('rental.csv', encoding='cp949', index_col=0)
bike_rental

,대여일자,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥),평일구분,대여소번호,대여소명,이용건수
0,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,162,봉원고가차도 밑,0
1,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,207,여의나루역 1번출구 앞,2
2,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,213,KT앞,2
3,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,434,신당 래미안 버스정류장,1
4,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,474,동대문역사문화공원역 1번출구 뒤편,5
...,...,...,...,...,...,...,...,...,...,...,...,...
87595,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,3758,서울남부출입국관리소,0
87596,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4009,월계프라자,0
87597,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4221,솔리스타오피스텔(공덕동주민센터),0
87598,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4591,신길자이아파트,1


In [24]:
# merge 기준이 될 열의 타입 변경 
bike_rental = bike_rental.astype({'대여일자':'object', '시간대':'int64', '대여소번호':'int64'})
bike_rental.dtypes

대여일자            object
시간대              int64
기온(°C)         float64
강수량(mm)        float64
풍속(m/s)        float64
습도(%)          float64
적설(cm)         float64
미세먼지농도(㎍/㎥)    float64
평일구분            object
대여소번호            int64
대여소명            object
이용건수             int64
dtype: object

In [25]:
total = pd.merge(bike_rental, bike_return, on=['대여일자', '시간대', '대여소번호'], how='left')
total

,대여일자,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥),평일구분,대여소번호,대여소명,이용건수,반납건수
0,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,162,봉원고가차도 밑,0,1.0
1,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,207,여의나루역 1번출구 앞,2,4.0
2,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,213,KT앞,2,2.0
3,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,434,신당 래미안 버스정류장,1,NaN
4,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,474,동대문역사문화공원역 1번출구 뒤편,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87595,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,3758,서울남부출입국관리소,0,NaN
87596,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4009,월계프라자,0,NaN
87597,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4221,솔리스타오피스텔(공덕동주민센터),0,NaN
87598,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4591,신길자이아파트,1,NaN


In [26]:
# 대여소명 열 삭제
total = total.drop(['대여소명'], axis=1)

# 반납건수가 null인 경우 0으로 채워주기
total['반납건수'].fillna(0, inplace=True)
total = total.astype({'반납건수':'int64'})

total.isnull().sum()

대여일자           0
시간대            0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
적설(cm)         0
미세먼지농도(㎍/㎥)    0
평일구분           0
대여소번호          0
이용건수           0
반납건수           0
dtype: int64

In [27]:
# 날씨 + 반납 데이터
weather_bike_return = total[['대여일자', '시간대', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', 
                             '적설(cm)', '미세먼지농도(㎍/㎥)', '평일구분', '대여소번호', '반납건수']]

weather_bike_return = weather_bike_return.rename(columns = {'대여일자':'반납일자'})
weather_bike_return

,반납일자,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥),평일구분,대여소번호,반납건수
0,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,162,1
1,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,207,4
2,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,213,2
3,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,434,0
4,2023-01-01,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,평일 외,474,1
...,...,...,...,...,...,...,...,...,...,...,...
87595,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,3758,0
87596,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4009,0
87597,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4221,0
87598,2023-12-31,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,평일 외,4591,0


In [30]:
# '월' 변수 만들기
weather_bike_return['반납일자'] = pd.to_datetime(weather_bike_return['반납일자'])
weather_bike_return['반납월'] = weather_bike_return['반납일자'].dt.month

# 평일구분 -> 평일로 바꾸기 (평일이면 1, 평일 아니면 0)
weather_bike_return['평일'] = 0
weather_bike_return['평일'] = np.where(weather_bike_return['평일구분'] == '평일', 1, 0) 

# 필요없는 열 삭제
weather_bike_return = weather_bike_return.drop(['평일구분', '반납일자'], axis=1)

weather_bike_return

,시간대,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),미세먼지농도(㎍/㎥),대여소번호,반납건수,반납월,평일
0,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,162,1,1,0
1,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,207,4,1,0
2,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,213,2,1,0
3,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,434,0,1,0
4,1,1.533333,0.000000,1.633333,73.333333,0.000000,84.333333,474,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
87595,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,3758,0,12,0
87596,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,4009,0,12,0
87597,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,4221,0,12,0
87598,8,1.700000,0.566667,2.033333,97.666667,6.566667,26.333333,4591,0,12,0


In [31]:
# 날씨 + 반납 데이터 내보내기
weather_bike_return.to_csv('2023년_따릉이반납_날씨_3시간별.csv', encoding='cp949')